In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os
from random import shuffle
from tqdm import tqdm


/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Reading labels

In [2]:
import pandas as pd
y=pd.read_csv('labels.csv')

### Loading Images and assigning them classes and resizing them

In [3]:
img_width = 120
img_height = 120
images=[]
classes=[]
#load training images
for i in range(120):
    for x in os.listdir("dog_train"):
        if i+1<=len(os.listdir('dog_train/'+x)) :
            img = cv2.imread('dog_train/'+x+'/'+os.listdir(os.path.join("dog_train",x))[i])
            resize_image = cv2.resize(img, (img_width,img_height))
            images.append(resize_image)
            classes.append(x)

### Creating one hot for classes

In [4]:
y = np.array(pd.get_dummies(classes,dtype="float32"))

## Image Augmentation
### This function will take a list of images and for each image there is 50% chance it will get passed to placeholder without any modification or it will get augmented in any of the 8 augmentations:  Flip (updown or left/right), Dilation, Blur, Rotation, Shear,Translation, RGB.

In [5]:
def augment(image):
    Aug_operation= ['flip_horz','flip_up','blur','dilation','rgb_','rotate','translate','sheer']
    img = []
    out=[]
    aug_img = []
    img_width = (image[0]).shape[0]
    img_height = (image[0]).shape[1]
    n_op = len(Aug_operation)
    for j in range(len(image)): 
        apply_aug =np.random.randint(0,2)
        if apply_aug == 0:
            aug_img = image[j]
        else:
            rand_int= np.random.randint(0,n_op)
            rows,cols,ch = image[0].shape 
            if rand_int%n_op == 0:
                aug_img = (cv2.flip(image[j],1))
            elif rand_int%n_op ==1:        
                aug_img=(cv2.flip(image[j],0))
            elif rand_int%n_op ==2:
                aug_img=(cv2.blur(image[j],(3,3)))
            elif rand_int%n_op ==3:
                kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
                aug_img=(cv2.dilate(image[j],kernel_ellipse,iterations=1))
            elif rand_int%n_op ==4:
                aug_img=cv2.cvtColor(image[j],cv2.COLOR_BGR2RGB)
            elif rand_int%n_op ==5:
                angle= np.random.uniform(-1,1)*90
                Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
                aug_img=(cv2.warpAffine(image[j],Rot_M,(cols,rows)))
            elif rand_int%n_op ==6:
                x_shift = np.random.uniform(-1,1)*img_width/8
                y_shift = np.random.uniform(-1,1)*img_height/8
                Trans_M = np.float32([[1,0,x_shift],[0,1,y_shift]])
                aug_img=(cv2.warpAffine(image[j],Trans_M,(cols,rows)))         
            elif rand_int%n_op ==7:           
                pts1 = np.float32([[5,5],[20,5],[5,20]])
                pts2 = np.float32([[10,10],[25,17],[8,20]])
                M = cv2.getAffineTransform(pts1,pts2)
                aug_img = cv2.warpAffine(image[j],M,(cols,rows))
        out.append(aug_img)
    return out

### Weight Initialization

In [6]:
def init_weight(shape):
    w=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return w

### Bias Initialization

In [7]:
def init_biases(shape):
    b=tf.Variable(tf.constant(0.1,shape=shape))
    return b

### Convolution

In [8]:
def conv2d(X1,w):
    return tf.nn.conv2d(X1,w,strides=[1,1,1,1],padding='SAME')

### Max Pooling

In [9]:
def max_pool(X1):
    return tf.nn.max_pool(X1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

### Applies Convolution -> Add Bias -> ReLU Activation 

In [10]:
def convolution_layer(X1,shape):
    w=init_weight(shape)
    b=init_biases([shape[3]])
    return tf.nn.relu(conv2d(X1,w)+b)

### Function for Fully Connected Layer

In [11]:
def fully_connected_layer(X1,size):
    feature=int(X1.get_shape()[1])
    w=init_weight([feature,size])
    b=init_biases([size])
    return tf.matmul(X1,w)+b

### Placeholder for Input and label (Image Size=[120,120])

In [12]:
X1=tf.placeholder(tf.float32,shape=[None,120,120,3])
y_true=tf.placeholder(tf.float32,shape=[None,120])

### Convolution(3,3,channel(3->16)) -> Max Pooling(2,2) -> Convolution(3,3,channel(16->32)) -> Max Pooling(2,2)

In [13]:
conv_1=convolution_layer(X1,shape=[3,3,3,16])
max_pool1=max_pool(conv_1)
conv_2=convolution_layer(max_pool1,shape=[3,3,16,32])
max_pool2=max_pool(conv_2)

In [14]:
# lrn1=tf.nn.local_response_normalization(
#     max_pool2,
#     depth_radius=5,
#     bias=1,
#     alpha=1,
#     beta=0.5,
#     name=None
# )

### Convolution(3,3,channel(32->64)) -> Max Pooling(2,2)

In [15]:
conv_3=convolution_layer(max_pool2,shape=[3,3,32,64])
max_pool3=max_pool(conv_3)
max_pool3

<tf.Tensor 'MaxPool_2:0' shape=(?, 15, 15, 64) dtype=float32>

### Convolution(3,3,channel(64->128))

In [16]:
conv_4=convolution_layer(max_pool3,shape=[3,3,64,128])
conv_4

<tf.Tensor 'Relu_3:0' shape=(?, 15, 15, 128) dtype=float32>

In [17]:
#max_pool4=max_pool(conv_4)

In [18]:
# lrn2=tf.nn.local_response_normalization(
#     max_pool4,
#     depth_radius=5,
#     bias=1,
#     alpha=1,
#     beta=0.5,
#     name=None
# )

In [19]:
#conv_5=convolution_layer(lrn2,shape=[3,3,64,128])
#max_pool5=max_pool(conv_5)
#conv_6=convolution_layer(max_pool5,shape=[3,3,128,256])
#max_pool6=max_pool(conv_6)
#hold_prob = tf.placeholder(tf.float32)
#full_one_dropout = tf.nn.dropout(max_pool6,keep_prob=hold_prob)

### Flatten

In [20]:
depth_flat=tf.reshape(conv_4,[-1,128*225])

### Fully Connected Layer(4096)->Batch_Normalization -> Fully Connected Layer(1024)->Dropout -> Prediction

In [21]:
fcn1=(fully_connected_layer(depth_flat,4096))
bn1=tf.nn.relu(tf.layers.batch_normalization(fcn1))
bn2=tf.nn.relu(bn1)
fcn2=tf.nn.relu(fully_connected_layer(bn2,1024))
hold_prob = tf.placeholder(tf.float32)
full_2_dropout = tf.nn.dropout(fcn2,keep_prob=hold_prob)
#fcn2=tf.nn.relu(fully_connected_layer(full_2_dropout,512))
#full_3_dropout = tf.nn.dropout(fcn2,keep_prob=hold_prob)
y_pred=(fully_connected_layer(full_2_dropout,120))
y_pred

<tf.Tensor 'add_6:0' shape=(?, 120) dtype=float32>

### Cross Entropy Loss and Training with Adam Optimizer

In [22]:
with tf.name_scope('Loss'):
    loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
with tf.name_scope('Adam'):
    train_op=tf.train.AdamOptimizer().minimize(loss_op)    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Accuracy

In [23]:
with tf.name_scope('Accuracy'):
    prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
    accuracy=tf.reduce_mean(tf.cast(prediction,tf.float32))

### Separating Training (9600) and Test Set (614)

In [24]:
images_train=images[:9600]
y_train=y[:9600]
images_test=images[9601:10214]
y_test=y[9601:10214]

### Graph and Summary

In [25]:
tf.summary.scalar("loss", loss_op)
tf.summary.scalar("accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()
saver=tf.train.Saver()

### Running the Session (11 Epochs and Batch Size=32).  training accuracy is very low after 11 epoch . Maximium Test Accuracy 5.05%

In [26]:
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter('./output1', sess.graph)
    steps=300
    sess.run(tf.global_variables_initializer())
    for j in range(60):        
        for i in range(steps):         
            batch_x=augment(images_train[32*i:32*(i+1)])
            batch_y=(y_train[32*i:32*(i+1)])            
            print(sess.run([train_op,loss_op,accuracy],feed_dict={X1:batch_x,y_true:batch_y,hold_prob:0.7}))
            _, c, summary = sess.run([train_op, loss_op, merged_summary_op],feed_dict={X1: batch_x, y_true: batch_y,hold_prob:0.7})
            summary_writer.add_summary(summary,i)
        print('epoch',j,' loss,accuracy:',sess.run([loss_op,accuracy],feed_dict={X1:images_test,y_true:y_test,hold_prob:1.0}))    

[None, 31004.656, 0.03125]
[None, 40337.78, 0.0]
[None, 20623.137, 0.0]
[None, 7510.3486, 0.0]
[None, 5740.366, 0.0]
[None, 3669.1575, 0.0]
[None, 2006.5256, 0.0]
[None, 1206.6278, 0.0]
[None, 745.177, 0.0]
[None, 555.466, 0.0]
[None, 388.38107, 0.03125]
[None, 399.5683, 0.0]
[None, 261.59317, 0.0]
[None, 187.86209, 0.0625]
[None, 199.77077, 0.0]
[None, 144.19896, 0.0]
[None, 126.12158, 0.0]
[None, 125.632034, 0.0]
[None, 99.91562, 0.03125]
[None, 84.73349, 0.03125]
[None, 77.14865, 0.0]
[None, 68.23395, 0.0]
[None, 83.097336, 0.0]
[None, 54.200127, 0.0]
[None, 45.981163, 0.0]
[None, 44.88315, 0.0]
[None, 51.87026, 0.03125]
[None, 39.418503, 0.0]
[None, 31.051905, 0.0]
[None, 23.924511, 0.0]
[None, 21.456364, 0.0]
[None, 24.242058, 0.0]
[None, 25.161766, 0.0]
[None, 16.217268, 0.0]
[None, 15.864899, 0.0]
[None, 12.838779, 0.0]
[None, 19.06913, 0.0]
[None, 14.81255, 0.0]
[None, 13.265144, 0.0]
[None, 10.291795, 0.0625]
[None, 7.279192, 0.03125]
[None, 13.2176285, 0.0]
[None, 13.357738, 

[None, 4.877826, 0.0]
[None, 4.785346, 0.0]
[None, 4.785337, 0.03125]
[None, 4.79813, 0.0]
[None, 4.840876, 0.0]
[None, 4.7877417, 0.0]
[None, 4.791554, 0.03125]
[None, 4.856003, 0.0]
[None, 4.86775, 0.0]
[None, 4.7845216, 0.03125]
[None, 4.9573727, 0.0]
[None, 4.7915688, 0.0]
[None, 4.7732363, 0.0]
[None, 4.7818685, 0.03125]
[None, 4.835062, 0.0]
[None, 4.791146, 0.0]
[None, 4.785422, 0.0]
[None, 5.145753, 0.03125]
[None, 4.7851486, 0.0]
[None, 4.784971, 0.0]
[None, 4.787803, 0.0]
[None, 4.9132795, 0.03125]
[None, 4.6783175, 0.0]
[None, 4.8429728, 0.0]
[None, 4.787731, 0.03125]
[None, 4.7905006, 0.0]
[None, 4.7915287, 0.0]
[None, 4.7850714, 0.0]
[None, 4.960381, 0.03125]
[None, 4.78911, 0.0]
[None, 4.791127, 0.0]
[None, 4.7854395, 0.0]
[None, 4.7857003, 0.03125]
[None, 4.787054, 0.0]
[None, 4.7900205, 0.0]
[None, 4.7877464, 0.0]
[None, 4.7914486, 0.03125]
[None, 4.7874975, 0.0]
[None, 4.7885933, 0.0]
[None, 4.7786236, 0.03125]
[None, 4.7606955, 0.0]
[None, 4.81131, 0.0]
[None, 4.78507

[None, 4.790919, 0.0]
[None, 4.8086157, 0.03125]
[None, 4.786004, 0.0]
[None, 4.7849755, 0.0]
[None, 4.7921705, 0.03125]
[None, 4.783139, 0.0]
[None, 4.788311, 0.0]
[None, 4.7949967, 0.0]
[None, 4.938942, 0.03125]
[None, 4.8631973, 0.0]
[None, 4.784186, 0.0]
[None, 4.864153, 0.0]
[None, 4.791596, 0.03125]
[None, 4.784908, 0.0]
[None, 4.785426, 0.0]
[None, 4.790864, 0.0]
[None, 4.7908154, 0.03125]
[None, 4.7860265, 0.0]
[None, 4.785061, 0.0]
[None, 4.792044, 0.03125]
[None, 4.783172, 0.0]
[None, 4.788317, 0.0]
[None, 4.7948904, 0.0]
[None, 4.7856083, 0.03125]
[None, 4.7907267, 0.0]
[None, 4.784221, 0.0]
[None, 4.781481, 0.0]
[None, 4.791503, 0.03125]
[None, 4.7849445, 0.0]
[None, 4.785431, 0.0]
[None, 4.790823, 0.0]
[None, 4.7943087, 0.03125]
[None, 4.7860413, 0.0]
[None, 4.9573913, 0.0]
[None, 4.8490453, 0.03125]
[None, 4.783232, 0.0]
[None, 4.788293, 0.0]
[None, 4.7947474, 0.0]
[None, 4.7672906, 0.0625]
[None, 4.790655, 0.0]
[None, 4.784256, 0.0]
[None, 4.7927113, 0.0]
[None, 4.775534

[None, 4.7905493, 0.03125]
[None, 4.786618, 0.0]
[None, 4.7848363, 0.0]
[None, 4.79319, 0.03125]
[None, 4.782117, 0.0]
[None, 5.067152, 0.0]
[None, 4.797036, 0.0]
[None, 4.7845845, 0.03125]
[None, 4.7926106, 0.0]
[None, 4.78384, 0.0]
[None, 4.7944765, 0.0]
[None, 4.791498, 0.03125]
[None, 5.188715, 0.0]
[None, 4.7994084, 0.0]
[None, 4.8546257, 0.0]
[None, 4.7832656, 0.0]
[None, 4.786678, 0.0]
[None, 4.7848434, 0.0]
[None, 4.7930436, 0.03125]
[None, 4.7821593, 0.0]
[None, 4.788824, 0.0]
[None, 4.7968388, 0.0]
[None, 4.784582, 0.03125]
[None, 4.7800317, 0.0]
[None, 4.783933, 0.0]
[None, 4.794342, 0.0]
[None, 4.7913656, 0.03125]
[None, 4.7853646, 0.0]
[None, 4.785279, 0.0]
[None, 4.7921267, 0.0]
[None, 4.7903767, 0.03125]
[None, 4.7867084, 0.0]
[None, 4.7848697, 0.0]
[None, 4.7929173, 0.03125]
[None, 4.782233, 0.0]
[None, 4.7887974, 0.0]
[None, 4.796651, 0.0]
[None, 4.7846174, 0.03125]
[None, 4.792385, 0.0]
[None, 4.7839847, 0.0]
[None, 4.7942076, 0.0]
[None, 4.791277, 0.03125]
[None, 4.7

[None, 4.786948, 0.0]
[None, 4.784767, 0.0]
[None, 4.793895, 0.03125]
[None, 4.781517, 0.0]
[None, 4.7889037, 0.0]
[None, 4.798339, 0.0]
[None, 4.7840967, 0.03125]
[None, 4.7935247, 0.0]
[None, 4.783855, 0.0]
[None, 4.7954297, 0.0]
[None, 4.791382, 0.03125]
[None, 4.785461, 0.0]
[None, 4.7853603, 0.0]
[None, 5.162714, 0.0]
[None, 4.7902765, 0.03125]
[None, 4.764475, 0.0]
[None, 4.784793, 0.0]
[None, 4.7937956, 0.03125]
[None, 4.7815876, 0.0]
[None, 4.7888165, 0.0]
[None, 4.798177, 0.0]
[None, 4.78415, 0.03125]
[None, 4.7933645, 0.0]
[None, 4.7838583, 0.0]
[None, 4.7953224, 0.0]
[None, 4.7912965, 0.03125]
[None, 4.7854414, 0.0]
[None, 4.7853456, 0.0]
[None, 4.793027, 0.0]
[None, 4.790163, 0.03125]
[None, 4.7868867, 0.0]
[None, 4.928157, 0.0]
[None, 4.7937064, 0.03125]
[None, 4.781658, 0.0]
[None, 4.7887545, 0.0]
[None, 4.7979894, 0.0]
[None, 4.784189, 0.03125]
[None, 4.7932386, 0.0]
[None, 4.7838926, 0.0]
[None, 4.8070135, 0.0]
[None, 4.7912016, 0.03125]
[None, 4.785468, 0.0]
[None, 4.7

...

[None, 4.8012934, 0.0]
[None, 4.7617702, 0.0]
[None, 5.103587, 0.03125]
[None, 4.7853756, 0.0]
[None, 4.7980127, 0.0]
[None, 4.791581, 0.0]
[None, 4.7879457, 0.0]
[None, 5.1016893, 0.0]
[None, 4.7983093, 0.0]
[None, 4.7792, 0.0]
[None, 4.7902737, 0.0]
[None, 4.805086, 0.0]
[None, 4.7830067, 0.0]
[None, 4.797929, 0.03125]
[None, 4.783024, 0.0]
[None, 4.801021, 0.0]
[None, 4.822419, 0.0]
[None, 4.78565, 0.03125]
[None, 4.7108073, 0.0]
[None, 4.797799, 0.0]
[None, 4.9323254, 0.0]
[None, 4.7879047, 0.0]
[None, 4.7984934, 0.0]
[None, 4.7822886, 0.0]
[None, 4.79921, 0.0]
[None, 4.790205, 0.0]
[None, 4.8046956, 0.0]
[None, 4.783011, 0.0]
[None, 4.678192, 0.0625]
[None, 4.7830734, 0.0]
[None, 4.800707, 0.0]
[None, 4.79315, 0.0]
[None, 4.778506, 0.03125]
[None, 4.798044, 0.0]
[None, 4.7975464, 0.0]
[None, 4.7912445, 0.0]
[None, 4.7878685, 0.0]
[None, 4.784695, 0.0]
[None, 4.7977962, 0.0]
[None, 4.7793646, 0.0]
[None, 4.79015, 0.0]
[None, 4.7802615, 0.0]
[None, 4.7854776, 0.0]
[None, 4.797493, 0

[None, 4.797353, 0.0]
[None, 4.790571, 0.0]
[None, 4.788725, 0.03125]
[None, 4.7843924, 0.0]
[None, 4.7977753, 0.0]
[None, 4.7785854, 0.03125]
[None, 4.791182, 0.0]
[None, 4.804342, 0.0]
[None, 4.782732, 0.0]
[None, 4.797572, 0.03125]
[None, 4.7836833, 0.0]
[None, 4.800012, 0.0]
[None, 4.792762, 0.0]
[None, 4.7859325, 0.03125]
[None, 4.785593, 0.0]
[None, 4.797123, 0.0]
[None, 4.79046, 0.0]
[None, 4.7886753, 0.03125]
[None, 4.7843914, 0.0]
[None, 4.797541, 0.0]
[None, 4.7786922, 0.03125]
[None, 4.7910786, 0.0]
[None, 4.8039904, 0.0]
[None, 4.7827673, 0.0]
[None, 4.7973785, 0.03125]
[None, 4.783699, 0.0]
[None, 4.7997394, 0.0]
[None, 4.7926188, 0.0]
[None, 4.7859244, 0.03125]
[None, 4.785571, 0.0]
[None, 4.796902, 0.0]
[None, 4.7903557, 0.0]
[None, 4.788623, 0.03125]
[None, 4.7843943, 0.0]
[None, 4.7973127, 0.0]
[None, 4.7787986, 0.03125]
[None, 4.7909813, 0.0]
[None, 4.803645, 0.0]
[None, 4.7828026, 0.0]
[None, 4.7971425, 0.03125]
[None, 4.7837186, 0.0]
[None, 4.7994704, 0.0]
[None, 4.

[None, 4.7903867, 0.0]
[None, 4.7884665, 0.03125]
[None, 4.7844543, 0.0]
[None, 4.7972646, 0.0]
[None, 4.8208075, 0.03125]
[None, 4.7906623, 0.0]
[None, 4.8036346, 0.0]
[None, 4.7827463, 0.0]
[None, 4.7971277, 0.03125]
[None, 4.7836037, 0.0]
[None, 4.799531, 0.0]
[None, 4.7924266, 0.0]
[None, 4.785915, 0.03125]
[None, 4.785405, 0.0]
[None, 4.796782, 0.0]
[None, 4.790289, 0.0]
[None, 4.7884083, 0.03125]
[None, 4.784458, 0.0]
[None, 4.7970495, 0.0]
[None, 4.779152, 0.03125]
[None, 4.7905626, 0.0]
[None, 4.803299, 0.0]
[None, 4.7827888, 0.0]
[None, 4.79692, 0.03125]
[None, 4.7836246, 0.0]
[None, 4.799268, 0.0]
[None, 4.7922955, 0.0]
[None, 4.6346674, 0.0625]
[None, 4.7853894, 0.0]
[None, 4.7965603, 0.0]
[None, 4.7902136, 0.0]
[None, 4.7883687, 0.03125]
[None, 4.784443, 0.0]
[None, 4.796801, 0.0]
[None, 4.779299, 0.03125]
[None, 4.8863473, 0.0]
[None, 4.802925, 0.0]
[None, 4.7827864, 0.0]
[None, 4.796772, 0.03125]
[None, 4.783683, 0.0]
[None, 4.7989507, 0.0]
[None, 4.79213, 0.0]
[None, 4.7

[None, 4.788019, 0.0]
[None, 4.784645, 0.0]
[None, 4.7966013, 0.0]
[None, 4.779625, 0.03125]
[None, 4.7900724, 0.0]
[None, 4.802929, 0.0]
[None, 4.7827115, 0.0]
[None, 4.796712, 0.03125]
[None, 4.7835793, 0.0]
[None, 4.79896, 0.0]
[None, 4.7921247, 0.0]
[None, 4.785882, 0.03125]
[None, 4.7853913, 0.0]
[None, 4.7962465, 0.0]
[None, 4.790285, 0.0]
[None, 4.7879744, 0.0]
[None, 4.7846518, 0.0]
[None, 4.796399, 0.0]
[None, 4.779724, 0.03125]
[None, 4.7899904, 0.0]
[None, 4.8026114, 0.0]
[None, 4.782756, 0.0]
[None, 4.796512, 0.03125]
[None, 4.7836084, 0.0]
[None, 4.7987127, 0.0]
[None, 4.7920027, 0.0]
[None, 4.7858763, 0.03125]
[None, 4.7853866, 0.0]
[None, 4.796048, 0.0]
[None, 4.7901945, 0.0]
[None, 4.787932, 0.0]
[None, 4.7846603, 0.0]
[None, 4.796202, 0.0]
[None, 4.779822, 0.03125]
[None, 4.7899113, 0.0]
[None, 4.8023014, 0.0]
[None, 4.782801, 0.0]
[None, 4.796317, 0.03125]
[None, 4.7836385, 0.0]
[None, 4.7984715, 0.0]
[None, 4.7918835, 0.0]
[None, 4.7858715, 0.03125]
[None, 4.785384, 

KeyboardInterrupt: 

In [ ]:
batch_x.shape

In [ ]:
train_data[0:50].shape

In [ ]:
train_data[:10]

In [ ]:
y_true.dtype

In [ ]:
X.shape